In [2]:
import smbus
import time
import numpy as np
from filterpy.kalman import KalmanFilter

In [18]:
"""This program handles the communication over I2C
between a Jetson Nano and a MPU-6050 Gyroscope / Accelerometer combo.
Made by: Dennis/TW
Released under the MIT License
Copyright 2019
"""
class mpu6050:
    # Global Variables
    GRAVITIY_MS2 = 9.80665
    address = 0x68
    bus = smbus.SMBus(0)
    # Scale Modifiers
    ACCEL_SCALE_MODIFIER_2G = 16384.0
    ACCEL_SCALE_MODIFIER_4G = 8192.0
    ACCEL_SCALE_MODIFIER_8G = 4096.0
    ACCEL_SCALE_MODIFIER_16G = 2048.0
    GYRO_SCALE_MODIFIER_250DEG = 131.0
    GYRO_SCALE_MODIFIER_500DEG = 65.5
    GYRO_SCALE_MODIFIER_1000DEG = 32.8
    GYRO_SCALE_MODIFIER_2000DEG = 16.4
    # Pre-defined ranges
    ACCEL_RANGE_2G = 0x00
    ACCEL_RANGE_4G = 0x08
    ACCEL_RANGE_8G = 0x10
    ACCEL_RANGE_16G = 0x18
    GYRO_RANGE_250DEG = 0x00
    GYRO_RANGE_500DEG = 0x08
    GYRO_RANGE_1000DEG = 0x10
    GYRO_RANGE_2000DEG = 0x18
    # MPU-6050 Registers
    PWR_MGMT_1 = 0x6B
    PWR_MGMT_2 = 0x6C
    SELF_TEST_X = 0x0D
    SELF_TEST_Y = 0x0E
    SELF_TEST_Z = 0x0F
    SELF_TEST_A = 0x10
    ACCEL_XOUT0 = 0x3B
    ACCEL_XOUT1 = 0x3C
    ACCEL_YOUT0 = 0x3D
    ACCEL_YOUT1 = 0x3E
    ACCEL_ZOUT0 = 0x3F
    ACCEL_ZOUT1 = 0x40
    TEMP_OUT0 = 0x41
    TEMP_OUT1 = 0x42
    GYRO_XOUT0 = 0x43
    GYRO_XOUT1 = 0x44
    GYRO_YOUT0 = 0x45
    GYRO_YOUT1 = 0x46
    GYRO_ZOUT0 = 0x47
    GYRO_ZOUT1 = 0x48
    ACCEL_CONFIG = 0x1C
    GYRO_CONFIG = 0x1B
    def __init__(self, address):
        self.address = address
        # Wake up the MPU-6050 since it starts in sleep mode
        self.bus.write_byte_data(self.address, self.PWR_MGMT_1, 0x00)
    # I2C communication methods
    def read_i2c_word(self, register):
        """Read two i2c registers and combine them.
        register -- the first register to read from.
        Returns the combined read results.
        """
        # Read the data from the registers
        high = self.bus.read_byte_data(self.address, register)
        low = self.bus.read_byte_data(self.address, register + 1)
        value = (high << 8) + low
        if (value >= 0x8000):
            return -((65535 - value) + 1)
        else:
            return value
    # MPU-6050 Methods
    def get_temp(self):
        """Reads the temperature from the onboard temperature sensor of the MPU-6050.
        Returns the temperature in degrees Celcius.
        """
        # Get the raw data
        raw_temp = self.read_i2c_word(self.TEMP_OUT0)
        # Get the actual temperature using the formule given in the
        # MPU-6050 Register Map and Descriptions revision 4.2, page 30
        actual_temp = (raw_temp / 340) + 36.53
        # Return the temperature
        return actual_temp
    def set_accel_range(self, accel_range):
        """Sets the range of the accelerometer to range.
        accel_range -- the range to set the accelerometer to. Using a
        pre-defined range is advised.
        """
        # First change it to 0x00 to make sure we write the correct value later
        self.bus.write_byte_data(self.address, self.ACCEL_CONFIG, 0x00)
        # Write the new range to the ACCEL_CONFIG register
        self.bus.write_byte_data(self.address, self.ACCEL_CONFIG, accel_range)
    def read_accel_range(self, raw=False):
        """Reads the range the accelerometer is set to.
        If raw is True, it will return the raw value from the ACCEL_CONFIG
        register
        If raw is False, it will return an integer: -1, 2, 4, 8 or 16. When it
        returns -1 something went wrong.
        """
        # Get the raw value
        raw_data = self.bus.read_byte_data(self.address, self.ACCEL_CONFIG)
        if raw is True:
            return raw_data
        elif raw is False:
            if raw_data == self.ACCEL_RANGE_2G:
                return 2
            elif raw_data == self.ACCEL_RANGE_4G:
                return 4
            elif raw_data == self.ACCEL_RANGE_8G:
                return 8
            elif raw_data == self.ACCEL_RANGE_16G:
                return 16
            else:
                return -1
    def get_accel_data(self, g=False):
        """Gets and returns the X, Y and Z values from the accelerometer.
        If g is True, it will return the data in g
        If g is False, it will return the data in m/s^2
        Returns a dictionary with the measurement results.
        """
        # Read the data from the MPU-6050
        x = self.read_i2c_word(self.ACCEL_XOUT0)
        y = self.read_i2c_word(self.ACCEL_YOUT0)
        z = self.read_i2c_word(self.ACCEL_ZOUT0)
        accel_scale_modifier = None
        accel_range = self.read_accel_range(True)
        if accel_range == self.ACCEL_RANGE_2G:
            accel_scale_modifier = self.ACCEL_SCALE_MODIFIER_2G
        elif accel_range == self.ACCEL_RANGE_4G:
            accel_scale_modifier = self.ACCEL_SCALE_MODIFIER_4G
        elif accel_range == self.ACCEL_RANGE_8G:
            accel_scale_modifier = self.ACCEL_SCALE_MODIFIER_8G
        elif accel_range == self.ACCEL_RANGE_16G:
            accel_scale_modifier = self.ACCEL_SCALE_MODIFIER_16G
        else:
            print("Unkown range - accel_scale_modifier set to self.ACCEL_SCALE_MODIFIER_2G")
            accel_scale_modifier = self.ACCEL_SCALE_MODIFIER_2G
        x = x / accel_scale_modifier
        y = y / accel_scale_modifier
        z = z / accel_scale_modifier
        if g is True:
            return {'x': x, 'y': y, 'z': z}
        elif g is False:
            x = x * self.GRAVITIY_MS2
            y = y * self.GRAVITIY_MS2
            z = z * self.GRAVITIY_MS2
            return {'x': x, 'y': y, 'z': z}
    def set_gyro_range(self, gyro_range):
        """Sets the range of the gyroscope to range.
        gyro_range -- the range to set the gyroscope to. Using a pre-defined
        range is advised.
        """
        # First change it to 0x00 to make sure we write the correct value later
        self.bus.write_byte_data(self.address, self.GYRO_CONFIG, 0x00)
        # Write the new range to the ACCEL_CONFIG register
        self.bus.write_byte_data(self.address, self.GYRO_CONFIG, gyro_range)
    def read_gyro_range(self, raw=False):
        """Reads the range the gyroscope is set to.
        If raw is True, it will return the raw value from the GYRO_CONFIG
        register.
        If raw is False, it will return 250, 500, 1000, 2000 or -1. If the
        returned value is equal to -1 something went wrong.
        """
        # Get the raw value
        raw_data = self.bus.read_byte_data(self.address, self.GYRO_CONFIG)
        if raw is True:
            return raw_data
        elif raw is False:
            if raw_data == self.GYRO_RANGE_250DEG:
                return 250
            elif raw_data == self.GYRO_RANGE_500DEG:
                return 500
            elif raw_data == self.GYRO_RANGE_1000DEG:
                return 1000
            elif raw_data == self.GYRO_RANGE_2000DEG:
                return 2000
            else:
                return -1
    def get_gyro_data(self):
        """Gets and returns the X, Y and Z values from the gyroscope.
        Returns the read values in a dictionary.
        """
        # Read the raw data from the MPU-6050
        x = self.read_i2c_word(self.GYRO_XOUT0)
        y = self.read_i2c_word(self.GYRO_YOUT0)
        z = self.read_i2c_word(self.GYRO_ZOUT0)
        gyro_scale_modifier = None
        gyro_range = self.read_gyro_range(True)
        if gyro_range == self.GYRO_RANGE_250DEG:
            gyro_scale_modifier = self.GYRO_SCALE_MODIFIER_250DEG
        elif gyro_range == self.GYRO_RANGE_500DEG:
            gyro_scale_modifier = self.GYRO_SCALE_MODIFIER_500DEG
        elif gyro_range == self.GYRO_RANGE_1000DEG:
            gyro_scale_modifier = self.GYRO_SCALE_MODIFIER_1000DEG
        elif gyro_range == self.GYRO_RANGE_2000DEG:
            gyro_scale_modifier = self.GYRO_SCALE_MODIFIER_2000DEG
        else:
            print("Unkown range - gyro_scale_modifier set to self.GYRO_SCALE_MODIFIER_250DEG")
            gyro_scale_modifier = self.GYRO_SCALE_MODIFIER_250DEG
        x = x / gyro_scale_modifier
        y = y / gyro_scale_modifier
        z = z / gyro_scale_modifier
        return {'x': x, 'y': y, 'z': z}
    def get_all_data(self):
        """Reads and returns all the available data."""
        temp = self.get_temp()
        accel = self.get_accel_data()
        gyro = self.get_gyro_data()
        return [accel, gyro, temp]


class giro:
    def init(self):
        self.mpu = mpu6050(0x68)
        self.mpu.set_accel_range(self.mpu.ACCEL_RANGE_2G)
        self.mpu.set_gyro_range(self.mpu.GYRO_RANGE_500DEG)
        self.offsets = self.offset_calculation()
    def get_z(self):
        gyro_data = self.mpu.get_gyro_data()
        #              # Угловая скорость
        gz = 1000 * gyro_data['z'] / 32768  - self.offsets[2]
        gz = round(gz / 1.72, 5)
        return gz


def offset_calculation():
    offsets = [[0.0],[0.0],[0.0]]
    mpu = mpu6050(0x68)
    mpu.set_accel_range(mpu.ACCEL_RANGE_2G)
    mpu.set_gyro_range(mpu.GYRO_RANGE_500DEG)
    buffer_x = []
    buffer_y = []
    buffer_z = []
    for i in range(100):
        # accel_data = mpu.get_gyro_data()
        # buffer_x.append(accel_data['x'])
        # buffer_y.append(accel_data['y'])
        # buffer_z.append(accel_data['z'])
        gyro_data = mpu.get_gyro_data()
                # Угловая скорость
        gx = 1000 * gyro_data['x'] / 32768
        gy = 1000 * gyro_data['y'] / 32768
        gz = 1000 * gyro_data['z'] / 32768
        buffer_x.append(gx)
        buffer_y.append(gy)
        buffer_z.append(gz)
    offsets[0] = sum(buffer_x) / 100
    offsets[1] = sum(buffer_y) / 100
    offsets[2] = sum(buffer_z) / 100
    # print(offsets)
    return offsets
    
# if __name__ == "__main__":
    # mpu = mpu6050(0x68)
    # mpu.set_accel_range(mpu.ACCEL_RANGE_2G)
    # mpu.set_gyro_range(mpu.GYRO_RANGE_500DEG)
    # print(mpu.get_temp())
    # offsets = offset_calculation()
    # a = 0.165
    # b = 0.140
    # time_old = time.time()
    # z = 0.0
    # # vx = 0.0
    # # vy = 0.0
    # # vz = 0.0
    # # my_filter = KalmanFilter(dim_x=2, dim_z=1)
    # # my_filter.x = np.array([[2.],[0.]])       # initial state (location and velocity)
    # # my_filter.F = np.array([[1.,1.],[0.,1.]])    # state transition matrix
    # # my_filter.H = np.array([[1.,0.]])    # Measurement function
    # # my_filter.P *= 1000.                 # covariance matrix
    # # my_filter.R = 5                      # state uncertainty
    # # my_filter.Q = 5 # process uncertainty
    # while 1:
    #     # accel_data = mpu.get_accel_data()
    #     # ax = accel_data['x'] - offsets[0]
    #     # ay = accel_data['y'] - offsets[1]
    #     # az = accel_data['z'] - offsets[2]
    #     # #ax = round(ax, 3)
    #     # ay = round(ay, 3)
    #     # az = round(az, 3)
    #     # #print("accel_data:" + "  " + str(ax) + "    " + str(ay) + "  " + str(az))
    #     # time_now = time.time()
    #     # my_filter.predict()
    #     # my_filter.update(ax)
    #     # # do something with the output
    #     # ax1 = my_filter.x[1][0]
    #     # #print(ax1, ax)
    #     # dt = time_now - time_old
    #     # vx += ax1 * dt
    #     # vy += vy + ay * dt
    #     # time_old = time_now
    #     # print("speed_data:" + "  " + str(round(vx, 3)) + "    " + str(ax))
    #     gyro_data = mpu.get_gyro_data()
    #              # Угловая скорость
    #     gx = 1000 * gyro_data['x'] / 32768  - offsets[0]
    #     gy = 1000 * gyro_data['y'] / 32768  - offsets[1]
    #     gz = 1000 * gyro_data['z'] / 32768  - offsets[2]
    #     gx = round(gx, 2)
    #     gy = round(gy, 2)
    #     gz = round(gz / 1.72, 3)
        
    #     time_now = time.time()
    #     dt = time_now - time_old
    #     z += gz * dt
    #     time_old = time_now
    #     # z = round(z, 5)
    #     # print("gyro:" + "  " + str(gx) + "   " + str(gy) + "  " + str(gz))
    #     # print(round(z, 3))
    #     # time.sleep(0.05)

In [16]:
while True:
    mpu = mpu6050(0x68)
    print("Temperature (C): ", mpu.get_temp())
    accel_data = mpu.get_accel_data()
    print("Acceleration x (m/s^2): ", accel_data['x'])
    print("Acceleration y (m/s^2): ", accel_data['y'])
    print("Acceleration z (m/s^2): ", accel_data['z'])
    gyro_data = mpu.get_gyro_data()
    print("Gyroscope x (deg/s): ", gyro_data['x'])
    print("Gyroscope y (deg/s): ", gyro_data['y'])
    print("Gyroscope z (deg/s): ", gyro_data['z'])
    time.sleep(1)

Temperature (C):  36.53
Acceleration x (m/s^2):  0.0
Acceleration y (m/s^2):  0.0
Acceleration z (m/s^2):  0.0
Gyroscope x (deg/s):  0.0
Gyroscope y (deg/s):  -2.618320610687023
Gyroscope z (deg/s):  -14.679389312977099
Temperature (C):  26.08294117647059
Acceleration x (m/s^2):  2.4516625
Acceleration y (m/s^2):  1.218648645019531
Acceleration z (m/s^2):  9.409212524414063
Gyroscope x (deg/s):  -3.312977099236641
Gyroscope y (deg/s):  0.6335877862595419
Gyroscope z (deg/s):  0.12213740458015267
Temperature (C):  26.224117647058826
Acceleration x (m/s^2):  1.8435352783203125
Acceleration y (m/s^2):  1.3096283081054687
Acceleration z (m/s^2):  9.744400756835937
Gyroscope x (deg/s):  -3.6106870229007635
Gyroscope y (deg/s):  0.030534351145038167
Gyroscope z (deg/s):  0.030534351145038167
Temperature (C):  26.36529411764706
Acceleration x (m/s^2):  2.0350714111328125
Acceleration y (m/s^2):  1.0438719238281249
Acceleration z (m/s^2):  9.507374792480467
Gyroscope x (deg/s):  -3.39694656488

KeyboardInterrupt: 

In [19]:
mpu = mpu6050(0x68)
mpu.set_accel_range(mpu.ACCEL_RANGE_2G)
mpu.set_gyro_range(mpu.GYRO_RANGE_500DEG)
print(mpu.get_temp())

27.118235294117646


In [20]:
mpu = mpu6050(0x68)
mpu.set_accel_range(mpu.ACCEL_RANGE_2G)
mpu.set_gyro_range(mpu.GYRO_RANGE_500DEG)
print(mpu.get_temp())
offsets = offset_calculation()
a = 0.165
b = 0.140
time_old = time.time()
z = 0.0
vx = 0.0
vy = 0.0
vz = 0.0
my_filter = KalmanFilter(dim_x=2, dim_z=1)
my_filter.x = np.array([[2.],[0.]])       # initial state (location and velocity)
my_filter.F = np.array([[1.,1.],[0.,1.]])    # state transition matrix
my_filter.H = np.array([[1.,0.]])    # Measurement function
my_filter.P *= 1000.                 # covariance matrix
my_filter.R = 5                      # state uncertainty
my_filter.Q = 5 # process uncertainty
while 1:
    accel_data = mpu.get_accel_data()
    ax = accel_data['x'] - offsets[0]
    ay = accel_data['y'] - offsets[1]
    az = accel_data['z'] - offsets[2]
    #ax = round(ax, 3)
    ay = round(ay, 3)
    az = round(az, 3)
    #print("accel_data:" + "  " + str(ax) + "    " + str(ay) + "  " + str(az))
    time_now = time.time()
    my_filter.predict()
    my_filter.update(ax)
    # do something with the output
    ax1 = my_filter.x[1][0]
    #print(ax1, ax)
    dt = time_now - time_old
    vx += ax1 * dt
    vy += vy + ay * dt
    time_old = time_now
    print("speed_data:" + "  " + str(round(vx, 3)) + "    " + str(ax))
    # gyro_data = mpu.get_gyro_data()
    #             # Угловая скорость
    # gx = 1000 * gyro_data['x'] / 32768  - offsets[0]
    # gy = 1000 * gyro_data['y'] / 32768  - offsets[1]
    # gz = 1000 * gyro_data['z'] / 32768  - offsets[2]
    # gx = round(gx, 2)
    # gy = round(gy, 2)
    # gz = round(gz / 1.72, 3)

    # time_now = time.time()
    # dt = time_now - time_old
    # z += gz * dt
    # time_old = time_now
    # z = round(z, 5)
    # print("gyro:" + "  " + str(gx) + "   " + str(gy) + "  " + str(gz))
    # print(round(z, 3))
    time.sleep(0.05)

27.165294117647058


NameError: name 'self' is not defined

In [11]:
tst = giro